<a href="https://colab.research.google.com/github/rupinchaudhry/ActivationFunctions/blob/main/ANNusngCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt



In [16]:
import time

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()


0

In [4]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [5]:
a=torch.FloatTensor([1.9, 2.7]).cuda()

In [6]:
a.device

device(type='cuda', index=0)

In [7]:
torch.cuda.memory_allocated()

512

In [8]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [10]:
df = pd.read_csv('iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)
labels = ['Iris setosa','Iris virginica','Iris versicolor','Mystery iris']

In [12]:
model=Model()

In [13]:
gpumodel=model.cuda()

In [14]:
next(gpumodel.parameters()).is_cuda

True

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [18]:
start = time.time()
#Train the model
epochs = 100
losses = []
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

end = time.time()-start
print(f"Total time {end:10.8f}")

epoch:  1  loss: 1.21604645
epoch: 11  loss: 0.89636081
epoch: 21  loss: 0.63588256
epoch: 31  loss: 0.42575309
epoch: 41  loss: 0.29078001
epoch: 51  loss: 0.18543366
epoch: 61  loss: 0.12307261
epoch: 71  loss: 0.09296677
epoch: 81  loss: 0.07825471
epoch: 91  loss: 0.07031446
Total time 0.73461652


In [19]:
#Evaluate the model
correct = 0
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f'{i+1:2}. {str(y_val):38}  {y_test[i]}')
        if y_val.argmax().item() == y_test[i]:
            correct += 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-3.5895,  3.5867, -1.3547], device='cuda:0')  1
 2. tensor([-2.2683,  4.6146, -2.8134], device='cuda:0')  1
 3. tensor([ 13.5709,   8.3669, -14.4077], device='cuda:0')  0
 4. tensor([-5.1572,  3.4121, -0.4987], device='cuda:0')  1
 5. tensor([-11.8691,   0.9443,   4.5598], device='cuda:0')  2
 6. tensor([-18.0286,  -0.8398,   8.7772], device='cuda:0')  2
 7. tensor([ 12.9466,   8.3620, -13.9767], device='cuda:0')  0
 8. tensor([ 15.1183,   9.2150, -15.9434], device='cuda:0')  0
 9. tensor([-11.5523,   1.3602,   4.0490], device='cuda:0')  2
10. tensor([-13.9034,   0.8091,   5.5644], device='cuda:0')  2
11. tensor([-15.3145,   0.2973,   6.6197], device='cuda:0')  2
12. tensor([ 13.4475,   8.1616, -14.1536], device='cuda:0')  0
13. tensor([-14.9512,   0.2268,   6.5068], device='cuda:0')  2
14. tensor([-5.9921,  2.9956,  0.2106], device='cuda:0')  1
15. tensor([-10.6659,   1.8166,   3.2899], device='cuda:0')  2
16. tensor([-2.3726,  4.3498, -2.5345], device='cuda:0')  1
17. ten